In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
import random, sys, os, multiprocessing
from itertools import chain
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, sem
from functools import partial

In [2]:
# function for computing cohen's D 
def _cohenD(df, parcel_name):
    dat = list(df[parcel_name])
    d = np.mean(dat)/np.std(dat, ddof = 1)  
    return d

# svr pipeline for each sampling bin 
def _svr_per_bin(merged_df, lab_type, parcel_list):
    
    """
    Main function that use SVR to measure a list of parcel's predictive accuracy for the selected beh performance
    
    Parameters:
    merged_df: a pandas dataframe that include neural and behaviral data. Column names include parcel name and beh measure names
    parcel_list: a list of parcel names (i.e., string), that will match the column names in merged_df
    lab_type: a string indicating the type of behaviral measure could be any of the following 5 strings: 
              1) WM_Task_2bk_Acc 2) ListSort_AgeAdj 3) PMAT24_A_CR 4) PicVocab_AgeAdj 5) ReadEng_AgeAdj
    
    Retrun:
    cur_bin_pred_acc: A single number that is the averaged of the 10 fold CV, measuring the predictive acc of the input parcel list. 
    
    """
    
    kf = KFold(n_splits=10, shuffle = False) # 10 fold
    feature = merged_df[parcel_list].to_numpy() # features of the current parcel_list
    label = merged_df[lab_type].to_numpy() # beh score

    pred_acc_list = [] 
    for train_index, test_index in kf.split(merge_df):

        train_feature, train_label = feature[train_index], label[train_index] # training feature and label
        test_feature, test_label = feature[test_index], label[test_index] # testing feature and label

        svr_pipeline = make_pipeline(StandardScaler(), SVR(kernel='linear',degree=1,
                                                       C=1.0,epsilon=0.1)) # svr pipeline
        svr_pipeline.fit(train_feature, train_label)  # train the model 
        pred_test_label = svr_pipeline.predict(test_feature) # test the model 
        pred_acc = pearsonr(test_label, pred_test_label)[0] # check pred acc 
        pred_acc_list.append(pred_acc) 
    
    cur_bin_pred_acc = np.mean(pred_acc_list) # mean of 10 folds
    return(cur_bin_pred_acc)

# randomly sample 10 parcels from the given bin list for N iterations. Getting N sampling data
def _sample_from_bin(parcel_bin, sample_size, iteration):
    
    """
    Function to do random sampling from each bin for N iteration 
    
    Parameters:
    bin_list: a list  containing the parcel names (i.e., string) of parcels in this bin
    iteration: int, how many samples to draw
    
    Retrun:
    iteration_sampling:  A list of all samplings. Each sampling is a list 10 parcels. 
    
    """
    
    seed_list = random.sample(range(1, 10000), iteration) # get seed
    if len(set(seed_list)) != iteration: # make sure seeds are unique
        print("seed duplication")  
    else:
        iteration_sampling = []
        for seed in seed_list: # for each given seed
            random.seed(seed)
            cur_iteration = random.sample(parcel_bin,sample_size)
            iteration_sampling.append(cur_iteration)
    
    return(iteration_sampling)


## predictive accuracy using full set features for each task (Gordon)

In [3]:
neural_data = pd.read_csv('/home/peetal/hulacon/nested_permutation/gordon333_cope11_rm_outlier.csv')

full_beh = pd.read_csv('/home/peetal/hulacon/nested_permutation/HCP_behavioral_data.csv') # behavioral
beh = full_beh[['Subject','WM_Task_2bk_Acc', 'ListSort_AgeAdj', 'PMAT24_A_CR', 'PicVocab_AgeAdj', 'ReadEng_AgeAdj']] # ID, in-scanner, and out-scanner task

# Effect size (Cohen's D) information
parcel_es = pd.DataFrame(columns = ['parcel_name','es']) # new df for cohen's D per parcel
parcel_es['parcel_name'] = list(neural_data.columns[1:])
parcel_es['es'] = [_cohenD(neural_data, parcel) for parcel in list(neural_data.columns[1:])]
parcel_es['abs_es'] = abs(parcel_es['es'])
parcel_es = parcel_es.sort_values(by=['abs_es'], ascending = False)

# join neural and behaviral data by subject id
merge_df = pd.merge(neural_data, beh, how='left', on=['Subject']).dropna() # join with ID

full_feature_acc = []
pool = multiprocessing.Pool(processes = 16)

for beh_lab in ['WM_Task_2bk_Acc', 'ListSort_AgeAdj', 'PMAT24_A_CR', 'PicVocab_AgeAdj', 'ReadEng_AgeAdj']:
    svr_per_shuffle_partial = partial(_svr_per_bin, merge_df, beh_lab)

    all_parcel_bin_list = list(parcel_es['parcel_name'])
    full_feature_acc.append(pool.map(svr_per_shuffle_partial, [all_parcel_bin_list]))

In [4]:
full_feature_acc

[[0.47225339973112324],
 [0.1901933952318764],
 [0.28288049279231486],
 [0.2598319542752106],
 [0.26017998357567096]]

## predictive accuracy using full set features for each task (Schaefer)

In [5]:
neural_data = pd.read_csv('/home/peetal/hulacon/nested_permutation/schaefer400_cope11_rm_outlier.csv') # neural
full_beh = pd.read_csv('/home/peetal/hulacon/nested_permutation/HCP_behavioral_data.csv') # behavioral
beh = full_beh[['Subject','WM_Task_2bk_Acc', 'ListSort_AgeAdj',
                'PMAT24_A_CR', 'PicVocab_AgeAdj', 'ReadEng_AgeAdj']] # ID, in-scanner, and out-scanner task

# Effect size (Cohen's D) information
parcel_es = pd.DataFrame(columns = ['parcel_name','es']) # new df for cohen's D per parcel
parcel_es['parcel_name'] = list(neural_data.columns[1:])
parcel_es['es'] = [_cohenD(neural_data, parcel) for parcel in list(neural_data.columns[1:])]
parcel_es['abs_es'] = abs(parcel_es['es'])
parcel_es = parcel_es.sort_values(by=['abs_es'], ascending = False)

# join neural and behaviral data by subject id
merge_df = pd.merge(neural_data, beh, how='left', on=['Subject']).dropna() # join with ID

full_feature_acc = []
pool = multiprocessing.Pool(processes = 16)

for beh_lab in ['WM_Task_2bk_Acc', 'ListSort_AgeAdj', 'PMAT24_A_CR', 'PicVocab_AgeAdj', 'ReadEng_AgeAdj']:
    svr_per_shuffle_partial = partial(_svr_per_bin, merge_df, beh_lab)

    all_parcel_bin_list = list(parcel_es['parcel_name'])
    full_feature_acc.append(pool.map(svr_per_shuffle_partial, [all_parcel_bin_list]))

In [6]:
full_feature_acc


[[0.451644173072573],
 [0.1776145396326645],
 [0.3068730618944142],
 [0.1994898404213258],
 [0.2641816550473497]]

In [ ]:
#def rewrite_df(df_dir):
    
#    full_dir = '/projects/hulacon/peetal/nested_permutation/result/' + df_dir
#    df = pd.read_csv(full_dir)
#    df['current_parcel'] = list(parcel_es['parcel_name'][0:60])
#    df['parcel_type'] = np.where(parcel_es['es'][0:60] > 0, "activated", "deactivated")
#    df['idx'] = list(range(1,61))

#    df.to_csv(full_dir, index = False)
    
#rewrite_df("schaefer_feature_selection_permutation_WM_Task_2bk_Acc.csv")
#rewrite_df("schaefer_feature_selection_permutation_ListSort_AgeAdj.csv")
#rewrite_df("schaefer_feature_selection_permutation_PicVocab_AgeAdj.csv")
#rewrite_df("schaefer_feature_selection_permutation_PMAT24_A_CR.csv")
#rewrite_df("schaefer_feature_selection_permutation_ReadEng_AgeAdj.csv")